Project Description

Objective: Use the GEDI L2B product to classify the San Pedro Riparian National Conservation Area (SPRNCA) by Foliage Height Diversity (FHD). 

FHD is an important metric for estimating ecological diversity and habitat suitability. Bird species are particularly sensitive in their habitat selection to vertical structure characteristics and the SPRNCA is an internationally recognized habitat zone for migratory and wintering birds. Zones would be classified using unsupervised techniques such as Agglomerative Clustering or DBSCAN. Once regions are classified,  I can evaluate present day trends in water stress over the different FHD classes to evaluate if parts of the corridor that are well suited for sensitive species habitat are particularly susceptible to water stress. To measure water stress, I will use either the ECOSTRESS Evaporative Stress Index (ESI) or EEflux’s METRIC based ET product coupled with ERA5’s PET estimate, depending on data availability. The primary outputs of this project will be maps classifying SPRNCA by FHD and time series plots and maps of the ESI for each FHD class.

The GEDI L2B product has an average spatial footprint of 25 meters and there are ~ 220 observations covering the SPRNCA corridor (I still need to determine how well this time series covers the SPRNCA). So this analysis would be limited to portions of the SPRNCA that have a riparian corridor wider than 25 meters. I also have near daily cloud masks from this area derived from Planet Labs imagery that I will use to filter the L2B and ESI products.

## Notes

GEDI data come in groups, and xarray can only read one at a time.

Quality FAQ with suggestions are here: https://lpdaac.usgs.gov/documents/589/GEDIL02_User_Guide_V1.pdf

conda install pyinterp -c conda-forge possible option for interpolation as it supports unstructured grids, 3D

this tweet has useful info on which beams use full power
https://twitter.com/GEDI_Knights/status/1228203192803966990

- BEAM0000 (ID 0) = Coverage
- BEAM0001 (ID 1) = Coverage
- BEAM0010 (ID 2) = Coverage
- BEAM0011 (ID 3) = Coverage
- BEAM0101 (ID 5) = Power
- BEAM0110 (ID 6) = Power
- BEAM1000 (ID 8) = Power
- BEAM1011 (ID 11) = Power


In [ ]:
import xarray as xr
import rioxarray as rx
import numpy as np
from pathlib import Path

In [ ]:
beam_groups = ["/BEAM0000", "/BEAM1011", "/BEAM1000", "/BEAM0110", "/BEAM0101", "/BEAM0011", "/BEAM0010", "/BEAM0001"]
beam_sub_groups = ["ancillary", "geolocation", "land_cover_data", "rx_processing"]
meta_group = "METADATA/DatasetIdentification"

In [ ]:
def open_beams(file_path, beam_groups = ["/BEAM0000", "/BEAM1011", "/BEAM1000", "/BEAM0110", "/BEAM0101", "/BEAM0011", "/BEAM0010", "/BEAM0001"]):
    datasets = []
    for beam in beam_groups:
        print(beam)
        # coord related values that need to be assigned to the dataset.
        ds_geo = xr.open_dataset(file_path, group=f"/{beam}/geolocation/")
        lat_coords = ds_geo.lat_highestreturn.values
        lon_coords = ds_geo.lon_highestreturn.values
        time_coords = ds_geo.delta_time.values
        
        # renaming and prooer meter units for z, vertical profile
        dz = xr.open_dataset(file_path, group=f"/{beam}/ancillary").dz.values
        ds = xr.open_dataset(file_path, group=beam)
        dim_names = list(ds.dims)
        ds = ds.rename({dim_names[1]:"z"}).drop_dims(dim_names[2]) # phony 6 only used for directional gap prob profile. 
        ds['z'] = ds.z * dz # make z have units in meters
        
        # assigning location and time coords to the variables depending on if they represent avg over profile or are taken along profile
        ds = ds.rename({dim_names[0]:"y"})
        ds['y'] = lat_coords
        ds = ds.assign({"x":lon_coords, "time":time_coords})
        
        datasets.append(ds)
    return datasets

In [ ]:
xr.open_dataset(file_paths[0], group="/BEAM1011/geolocation")

In [ ]:
file_paths = list(Path("data").glob("*.h5"))

In [ ]:
list(xr.open_dataset(file_paths[0], group="/BEAM1011").dims)

In [ ]:
open_beams(file_paths[0])

In [ ]:
test_beam.sensitivity
test_beam.fhd_normal
test_beam.l2a_quality_flag
test_beam.l2b_quality_flag
test_beam.pai
test_beam.pai_z
test_beam.pavd_z
test_beam.l2b_quality_flag

In [ ]:
ds = xr.Dataset({'pai_z': (['x', 'y', 'time', 'z'],  test_beam.pai_z),
                 coords={'x': (['x', 'y'], lon_coords),
                         'y': (['x', 'y'], lat_coords),
                         'time': time_coords,
                        "z": test_beam.phony_dim_5.values}})